In [1]:
import pandas as pd
import sys, os , time
from tabulate import tabulate

def find_match(string,list_des):
    return [value for value in list_des if string in value]

def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(folder+file_name)
    #rint(table)
    print(tabulate(table, headers='keys', tablefmt='rst')) #latex

In [2]:
!ls

aux.txt		 Plot_Results.ipynb  simple
calculate_mv.py  run_CIFAR.sh	     simple_w_reset
CIFAR		 run_simple.sh	     simple_w_softinit


In [3]:
!ls simple

result_scenario1.txt		 synthetic_Ours2_trainAnn_s2.csv
result_scenario2.txt		 synthetic_Ours2_trainAnn_s3.csv
result_scenario3.txt		 synthetic_Ours2_trainAnn_s4.csv
result_scenario4.txt		 synthetic_Ours2_trainAnn_s5.csv
result_scenario5.txt		 synthetic_Ours2_trainAnn_s6.csv
result_scenario6.txt		 synthetic_Ours2_train_s1.csv
synthetic_DS_test_s1.csv	 synthetic_Ours2_train_s2.csv
synthetic_DS_test_s2.csv	 synthetic_Ours2_train_s3.csv
synthetic_DS_test_s3.csv	 synthetic_Ours2_train_s4.csv
synthetic_DS_test_s4.csv	 synthetic_Ours2_train_s5.csv
synthetic_DS_test_s5.csv	 synthetic_Ours2_train_s6.csv
synthetic_DS_test_s6.csv	 synthetic_Ours3_testAux_s1.csv
synthetic_DS_train_s1.csv	 synthetic_Ours3_testAux_s2.csv
synthetic_DS_train_s2.csv	 synthetic_Ours3_testAux_s3.csv
synthetic_DS_train_s3.csv	 synthetic_Ours3_testAux_s4.csv
synthetic_DS_train_s4.csv	 synthetic_Ours3_testAux_s5.csv
synthetic_DS_train_s5.csv	 synthetic_Ours3_testAux_s6.csv
synthetic_DS_train_s6.csv	 synthetic_Ours3_test_s1.

In [71]:
folder = "simple_w_softinit/"
files = sorted(os.listdir(folder))

In [72]:
for s in range(1,10):
    print("\n****************************** Scenario "+str(s)+" ******************************")
    scenario_files = find_match("s"+str(s)+".csv",files)
    
    #all from softMV
    aux = find_match("softMV",scenario_files)
    for value in aux:
        plot_files(value)

    #all from hardMV
    aux = find_match("hardMV",scenario_files)
    for value in aux:
        plot_files(value)
        
    aux = find_match("DS",scenario_files)
    for value in aux:
        plot_files(value)
        
    aux = find_match("Raykar",scenario_files)
    for value in aux:
        plot_files(value)
        
    aux = find_match("Ours1",scenario_files)
    for value in aux:
        plot_files(value)        
    aux = find_match("Ours2",scenario_files)
    for value in aux:
        plot_files(value)
    aux = find_match("Ours3",scenario_files)
    for value in aux:
        plot_files(value)


****************************** Scenario 1 ******************************

File to show:  synthetic_Ours1_testAux_s1.csv
====  ========  =========  ===========  =========  ==================  ==========  ============
  ..    Groups    Entropy    Diag Mean    KL to I    I similar % (JS)    Accuracy    F1 (micro)
====  ========  =========  ===========  =========  ==================  ==========  ============
   0         0   0.341519     0.816094   0.385683            0.872727    0.769841      0.769841
   1         1   0.368553     0.790594   0.380607            0.859325    0.73431       0.73431
   2         2   0.332379     0.754149   0.608073            0.814966    0.695238      0.695238
====  ========  =========  ===========  =========  ==================  ==========  ============

File to show:  synthetic_Ours1_test_s1.csv
====  ============  ==========  ============
  ..  Unnamed: 0      Accuracy    F1 (micro)
====  ============  ==========  ============
   0  Global          0.88754

====  ============  ==========  ============  ============  =====================  ==============
  ..  Unnamed: 0      Accuracy    F1 (micro)    Average KL    Average PearsonCorr    Average RMSE
====  ============  ==========  ============  ============  =====================  ==============
   0  Global          0.800326      0.800326      0.728658               0.250575        0.191271
====  ============  ==========  ============  ============  =====================  ==============

File to show:  synthetic_Ours3_testAux_s6.csv
====  ========  =========  ===========  =========  ==================  ==========  ============
  ..    Groups    Entropy    Diag Mean    KL to I    I similar % (JS)    Accuracy    F1 (micro)
====  ========  =========  ===========  =========  ==================  ==========  ============
   0         0   0.252323     0.720892  0.766237             0.771707    0.697131      0.697131
   1         1   0.24544      0.87714   0.225756             0.918812    0.8172